<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 6: Использование Advantage Actor Critic (A2C) для моделирования робототехники с помощью PyBullet и Panda-Gym 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail" width="50%"/>

В этом блокноте вы научитесь использовать A2C с двумя наборами робототехнических сред PyBullet и Panda-Gym.

С [PyBullet](https://github.com/bulletphysics/bullet3), мы собираемся **обучить робота двигаться**:
- `AntBulletEnv-v0` 🕸️ Точнее, паук (они говорят Муравей, но да ладно... это паук 😆 ) 🕸️

Затем, с [Panda-Gym](https://github.com/qgallouedec/panda-gym), мы собираемся **обучить роботизированную руку** (Франка Эмика Панда робот) выполнять задание:
- `Reach`: робот должен поместить свой конечный манипулятор в целевое положение.

После этого вы сможете **тренироваться в других средах робототехники**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/environments.gif" alt="Robotics environments" width="50%"/>

### 🎮 Окружающая среда: 

- [PyBullet](https://github.com/bulletphysics/bullet3)
- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

###📚 RL-Библиотека: 

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

## Цели этого блокнота 🏆

В конце этого блокнота вы:

- Научитесь использовать библиотеки сред **PyBullet** и **Panda-Gym**.
-  Научитесь **обучать роботов, используя A2C**.
- Поймите, почему **нам нужно нормализовать входные данные**.
- Будьте в состоянии **отправить своего обученного агента и код в центр** с хорошим видеоповтором и оценочным баллом 🔥.

## Этот блокнот взят из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

🔲 📚 [Метод "актер-критик" Блок 6](https://huggingface.co/deep-rl-course/unit6/introduction) 🤗  

# Давайте обучим наших первых роботов 🤖

Чтобы подтвердить это практическое руководство для [процесса сертификации](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process),  вам нужно отправить две ваши обученные модели в концентратор и получить следующие результаты:

- `AntBulletEnv-v0` получив результат >= 650.
- `PandaReachDense-v2` получив результат >= -3.5.

Чтобы найти свой результат, перейдите к [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) и найдите свою модель, **the result = mean_reward - std of reward**

Если вы не нашли свою модель, **перейдите в нижнюю часть страницы и нажмите на кнопку обновить**

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process